In [99]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install pandas matplotlib

In [100]:
import pandas as pd

In [101]:
df = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
df

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
...,...,...,...,...,...
44863,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44864,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44865,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44866,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True


### Creazione di train e test

In [102]:
df = df.drop('source',axis = 1)
df = df.drop('RDizzl3_seven',axis = 1)
data = df
data = data.drop('label',axis = 1)
data = data.drop('prompt_name',axis = 1)

In [103]:
data

,text
0,Phones\n\nModern humans today are always on th...
1,This essay will explain if drivers should or s...
2,Driving while the use of cellular devices\n\nT...
3,Phones & Driving\n\nDrivers should not be able...
4,Cell Phone Operation While Driving\n\nThe abil...
...,...
44863,"Dear Senator,\n\nI am writing to you today to ..."
44864,"Dear Senator,\n\nI am writing to you today to ..."
44865,"Dear Senator,\n\nI am writing to you today to ..."
44866,"Dear Senator,\n\nI am writing to you today to ..."


In [104]:
lunghezza_massima = max(len(sequenza.split()) for sequenza in df['text'])

print("Lunghezza massima delle sequenze:", lunghezza_massima)

Lunghezza massima delle sequenze: 1656


In [105]:
import math
# Calcola la lunghezza media delle sequenze nel dataset
lunghezza_medio = sum(len(sequenza.split()) for sequenza in df['text']) / len(df['text'])
lunghezza_medio = math.ceil(lunghezza_medio)
print("Lunghezza media delle sequenze:", lunghezza_medio)

Lunghezza media delle sequenze: 384


In [106]:
import numpy as np
# Calcola la lunghezza delle sequenze
lunghezze_sequenze = [len(sequenza.split()) for sequenza in df['text']]

# Scegli il percentile desiderato (ad esempio, il 95° percentile)
percentile_desiderato = 95

# Calcola il valore del percentile
lunghezza_percentile = np.percentile(lunghezze_sequenze, percentile_desiderato)

lunghezza_percentile =  math.ceil(lunghezza_percentile)

print(f"{percentile_desiderato}° percentile della lunghezza delle sequenze:", lunghezza_percentile)

95° percentile della lunghezza delle sequenze: 713


In [107]:
from sklearn.model_selection import train_test_split

# Supponiamo che il tuo dataset sia rappresentato da X (feature) e y (etichette)
X_train, X_temp, y_train, y_temp = train_test_split(data['text'], df['label'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Create text encoder


In [108]:
import tensorflow as tf

In [109]:
VOCAB_SIZE = 1000 #Parametro da sistemare 
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE) #output_sequence_length=50
encoder.adapt(X_train.values.flatten())

In [110]:
#X_train.head() non serve perchè lo fa la rete

In [111]:
#X_train = encoder(X_train) non serve perchè lo fa la rete

In [112]:
#X_train non serve perchè lo fa la rete

In [113]:
import numpy as np

##### Controlliamo se l'encoder funziona 

In [114]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'to', 'and', 'a', 'of', 'in', 'that', 'is',
       'it', 'for', 'be', 'you', 'can', 'they', 'are', 'have', 'students',
       'i'], dtype='<U15')

### Applichiamo il Padding (VEDERE MEGLIO COME FARE QUESTA COSA)


In [115]:
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#X_train = pad_sequences(X_train, padding='post', maxlen=100)

In [116]:
#X_train

### Creiamo un embedding

In [117]:
embedding_layer = tf.keras.layers.Embedding(1000, 5) #SERVE SOLO DA ESEMPIO IL VERO PADDING è NELLA RETE

In [118]:
#TEST PER VEDERE SE FUNZIONA IL LAYER EMBEDDING #SERVE SOLO DA ESEMPIO IL VERO PADDING è NELLA RETE
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.03891827,  0.0376831 ,  0.00992774, -0.03390513, -0.00582019],
       [ 0.0232645 , -0.0119755 ,  0.03679414, -0.04613665,  0.00498061],
       [-0.02000946,  0.00447591, -0.02872385,  0.03134989, -0.03945293]],
      dtype=float32)

## Creazione modello

In [121]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        #gestire anche dimensione input e output
        input_dim=len(encoder.get_vocabulary()),
        output_dim=3,
        # aggiungere input_length  =  lunghezza massima o come media o come percentile 95%
        input_length = lunghezza_massima,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [122]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4), # default 0.01
              metrics=['accuracy'])

In [123]:
history = model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 586/1122 [==============>...............] - ETA: 3:10 - loss: 0.1132 - accuracy: 0.9661

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
for key in X_test :
    predictions = model.predict(np.array([key]))
    print(predictions)
    break

In [ ]:
for key in y_test :
    print(key)
    break